In [51]:
!pip install PyGithub
!pip install transformers

from github import Github
from transformers import pipeline
import time

pipe = pipeline("text2text-generation", model="microsoft/codereviewer")

In [43]:
TOKEN = ""  # Replace with your GitHub Personal Access Token
REPO_NAME = "JetBrains/intellij-community"  # Replace with the desired repository name "<user>/<repo name>"
PR_NUMBER = 2569  # Replace with the desired pull request number (for spot checking )
g = Github(TOKEN)
repo = g.get_repo(REPO_NAME)

In [19]:
def collect_review_data(max_pulls=10):
    pull_requests = repo.get_pulls(state='closed', sort='updated', direction='desc')

    review_diffs = []
    review_comments = []

    for pr in pull_requests:
        print(pr)
        for comment in pr.get_review_comments():
            print(comment)
            review_diffs.append(comment.diff_hunk)
            review_comments.append(comment.body)

    return review_diffs, review_comments

review_sample_diffs, review_sample_comments = collect_review_data()

In [44]:
# Get review comments for the specific pull request
def collect_review_comments_from_pr(pr_number):
    pr = repo.get_pull(pr_number)

    review_diffs = []
    review_comments = []

    for comment in pr.get_review_comments():
        review_diffs.append(comment.diff_hunk)
        review_comments.append(comment.body)

    return review_diffs, review_comments

pr_sample_diff, pr_sample_comment = collect_review_comments_from_pr(PR_NUMBER)


In [57]:
code_diff = pr_sample_diff[0]
print('#### Code: \n ``` \n', code_diff, '\n ```')
print('#### Original Comment: \n',pr_sample_comment[0])
print('#### CodeReviewer Comment:')
pipe(code_diff, max_length=512, num_return_sequences=1)[0]["generated_text"] # does not work properly in the loop

#### Code: 
 ``` 
 @@ -37,16 +37,22 @@ open class AddModifierFix(
     protected fun invokeOnElement(element: KtModifierListOwner?) {
         element?.addModifier(modifier)
 
-        if (modifier == KtTokens.ABSTRACT_KEYWORD && (element is KtProperty || element is KtNamedFunction)) {
-            element.containingClass()?.run {
-                if (!hasModifier(KtTokens.ABSTRACT_KEYWORD) && !hasModifier(KtTokens.SEALED_KEYWORD)) {
-                    addModifier(KtTokens.ABSTRACT_KEYWORD)
+        when (modifier) {
+            KtTokens.ABSTRACT_KEYWORD -> {
+                if (element is KtProperty || element is KtNamedFunction) {
+                    element.containingClass()?.run {
+                        if (!hasModifier(KtTokens.ABSTRACT_KEYWORD) && !hasModifier(KtTokens.SEALED_KEYWORD)) {
+                            addModifier(KtTokens.ABSTRACT_KEYWORD)
+                        }
+                    }
                 }
             }
-        }
 
-        if (modifier =

"<msg>This is a good change, but I think it's a good idea to add a `TODO` here."